In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.identity_mapping = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.identity_mapping = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        identity = self.identity_mapping(x)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = x + identity
        x = self.relu(x)
        return x

In [3]:
class ResNet34(nn.Module):
    def __init__(self, num_classes, blocks):
        super(ResNet34, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.layer1 = self.make_layer(in_channels=64, out_channels=64, num_blocks=blocks[0], stride=1)
        self.layer2 = self.make_layer(in_channels=64, out_channels=128, num_blocks=blocks[1], stride=2)
        self.layer3 = self.make_layer(in_channels=128, out_channels=256, num_blocks=blocks[2], stride=2)
        self.layer4 = self.make_layer(in_channels=256, out_channels=512, num_blocks=blocks[3], stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(Block(in_channels, out_channels, stride))  # Downsampling block
        for _ in range(1, num_blocks):
            layers.append(Block(out_channels, out_channels))  # Identity blocks
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x

In [4]:
from scripts.cv_utils import *

In [5]:
transform = cifar_transform()
train_loader, test_loader = download_cifar100(transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
# hyperparameters
n_epochs = 2
save_checkpoint = 1
num_classes = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Current device for training:", device)
print("#" * 20)

model = ResNet34(num_classes=num_classes, blocks=[3, 4, 6, 3])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)

Current device for training: cuda
####################


In [7]:
from scripts.cv_utils import *

epoch_loss_history, batch_loss_history = train(model, train_loader, test_loader, optimizer, criterion, device, model_name="res_net", )

Training model on device:  cuda
Epoch 1/10: Train Loss: 3.8309 - Val Loss: 3.2087
    Train Acc: 10.92% - Val Acc: 12.50%
####################
Saving model checkpoint to models/res_net/checkpoint_0.0.pth
Epoch 2/10: Train Loss: 2.8432 - Val Loss: 2.5862
    Train Acc: 28.00% - Val Acc: 31.25%
####################
Saving model checkpoint to models/res_net/checkpoint_1.0.pth
Epoch 3/10: Train Loss: 2.2511 - Val Loss: 2.2587
    Train Acc: 40.28% - Val Acc: 50.00%
####################
Saving model checkpoint to models/res_net/checkpoint_2.0.pth
Epoch 4/10: Train Loss: 1.8381 - Val Loss: 2.2753
    Train Acc: 49.76% - Val Acc: 31.25%
####################
Saving model checkpoint to models/res_net/checkpoint_3.0.pth


KeyboardInterrupt: 

In [24]:
class BottleNeck(nn.Module):
    def __init__(self, in_channels, out_channels, bottleneck_channels=None, stride=1, downsample=None):
        super(BottleNeck, self).__init__()
        
        if bottleneck_channels is not None:
            mid_channels = bottleneck_channels
        else:
            mid_channels = out_channels // 4
        
        self.conv1 = nn.Conv2d(in_channels, mid_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(mid_channels)
        self.conv2 = nn.Conv2d(mid_channels, mid_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(mid_channels)
        self.conv3 = nn.Conv2d(mid_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.identity_mapping = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.identity_mapping = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = self.identity_mapping(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out += identity
        out = self.relu(out)

        return out

In [25]:
x = torch.randn(64, 3, 224, 224)
conv1 = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

x = conv1(x)
print(x.shape)

torch.Size([64, 64, 56, 56])


In [26]:
in_channels = 64
out_channels = 256
stride = 1
num_blocks = 3

layers = []
layers.append(BottleNeck(in_channels, out_channels, bottleneck_channels=64, stride=stride))  # Downsampling block
for _ in range(1, num_blocks):
    layers.append(BottleNeck(out_channels, out_channels, bottleneck_channels=64))  # Identity blocks


layer1 = nn.Sequential(*layers)

In [27]:
x = layer1(x)
print(x.shape)

torch.Size([64, 256, 56, 56])


In [28]:
in_channels = 256
out_channels = 512
stride = 2
num_blocks = 4

layers = []
layers.append(BottleNeck(in_channels, out_channels, stride=stride, bottleneck_channels=128))  # Downsampling block
for _ in range(1, num_blocks):
    layers.append(BottleNeck(out_channels, out_channels, bottleneck_channels=128))  # Identity blocks

layer1 = nn.Sequential(*layers)

In [29]:
x = layer1(x)
print(x.shape)

torch.Size([64, 512, 28, 28])


In [36]:
class ResNet50(nn.Module):
    def __init__(self, num_classes=100):
        super(ResNet50, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        self.layer1 = self.make_layer(64, 256, 64, stride=1, num_blocks=3)
        self.layer2 = self.make_layer(256, 512, 128, stride=2, num_blocks=4)
        self.layer3 = self.make_layer(512, 1024, 256, stride=2, num_blocks=6)
        self.layer4 = self.make_layer(1024, 2048, 512, stride=2, num_blocks=3)
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048, num_classes)

    def make_layer(self, in_channels, out_channels, bottleneck_channels, stride, num_blocks):
        layers = []
        layers.append(BottleNeck(in_channels, out_channels, stride=stride, bottleneck_channels=bottleneck_channels))  
        for _ in range(1, num_blocks):
            layers.append(BottleNeck(out_channels, out_channels, bottleneck_channels=bottleneck_channels)) 
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

In [37]:
x = torch.randn(64, 3, 224, 224)
model = ResNet50(100)

x = model(x)
print(x.shape)

torch.Size([64, 100])
